In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

In [ ]:
# 1) Carregar e preparar a série temporal mensal
df17 = pd.read_csv('/IMP_2017.csv', sep=';')
df18 = pd.read_csv('/IMP_2018.csv', sep=';')
df19 = pd.read_csv('/IMP_2019.csv', sep=';')
df20 = pd.read_csv('/IMP_2020.csv', sep=';')
df21 = pd.read_csv('/IMP_2021.csv', sep=';')
df22 = pd.read_csv('/IMP_2022.csv', sep=';')
df23 = pd.read_csv('/IMP_2023.csv', sep=';')
df = pd.concat([df17, df18, df19, df20, df21, df22, df23], ignore_index=True)
df = df[(df['CO_NCM'] == 84713012)].copy()
df['CO_ANO'] = df['CO_ANO'].astype(int)
df['CO_MES'] = df['CO_MES'].astype(int)
df['date'] = pd.to_datetime(df['CO_ANO'].astype(str) + '-' + df['CO_MES'].astype(str) + '-01')
ts = df.groupby('date')['KG_LIQUIDO'].sum().sort_index()

In [ ]:
# 2) Normalizar (MinMax)
values = ts.values.reshape(-1, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

# 3) Gerar janelas (window = 12 meses)
window = 12
X, y = [], []
for i in range(window, len(scaled)):
    X.append(scaled[i-window:i, 0])
    y.append(scaled[i, 0])
X, y = np.array(X), np.array(y)

# 4) Reshape para [samples, timesteps, features]
X = X.reshape((X.shape[0], X.shape[1], 1))

# 5) Separar treino e teste (80% treino)
split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# 6) Definir o modelo LSTM
model = Sequential([
    LSTM(50, activation='tanh', input_shape=(window, 1)),
    LSTM(25, activation='tanh', input_shape=(window, 1)),
    Dense(1)
])
model.compile(optimizer='adam', loss='mse', metrics=['accuracy'])

# 7) Treinar
history = model.fit(
    X_train, y_train,
    epochs=100,
    batch_size=1,
    validation_data=(X_test, y_test),

    verbose=2
)

# 8) Previsão e inversão de escala
y_pred = model.predict(X_test)
y_pred_inv = scaler.inverse_transform(y_pred)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

# 9) Plotar resultados
plt.figure(figsize=(8, 4))
plt.plot(ts.index[window + split:], y_test_inv, label='Real')
plt.plot(ts.index[window + split:], y_pred_inv, label='Previsto', linestyle='--')
plt.title('LSTM: Previsão de KG_LIQUIDO (NCM 84713012)')
plt.xlabel('Data')
plt.ylabel('KG Líquido')
plt.legend()
plt.tight_layout()
plt.show()



In [ ]:

scoreCkpt = model.evaluate(y_train, y_test, verbose=0)